In [1]:
%load_ext autoreload
%autoreload 2
# print current working directory
import os
# cd to the previous directory
# os.chdir('..')
# if not 'bertviz_repo' in sys.path:
#   print('Adding bertviz_repo to sys.path')
#   sys.path += ['bertviz_repo']
print(os.getcwd())

/local/home/isiddig/Desktop/uds/thesis/project-code


In [2]:
from smiles_spe_tokenizer import SMILES_SPE_Tokenizer
import datasets
import torch
import sys
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import pipeline
from transformers import RobertaConfig, RobertaForMaskedLM, LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer , DataCollatorWithPadding , AutoTokenizer, AutoConfig , AutoModelForCausalLM
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline
from datasets import Dataset
from datasets import load_dataset, load_metric
import os
import wandb
import torch
import evaluate
from bertviz_repo.bertviz import head_view
from CollatorDatasetTrainer import CustomDataCollator
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from CollatorDatasetTrainer import CustomDataCollator , ToxicTrainer
import numpy as np
from utils import *
from utils import split_stratified_into_train_val_test
from datasets import load_metric
from captum.attr import LayerIntegratedGradients
from captum.attr import visualization as viz
from sklearn.metrics import balanced_accuracy_score , accuracy_score , precision_score , recall_score , f1_score , roc_auc_score
from bert_loves_chemistry.chemberta.utils.molnet_dataloader import load_molnet_dataset, write_molnet_dataset_for_chemprop


2023-07-04 01:33:47.915341: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/isiddig/anaconda3/envs/toxicity/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
# prepare the tokenizers
Spe_tokenizer = SMILES_SPE_Tokenizer(vocab_file='vocabs/spe_vocab/vocab_speSMILESDB_1000.txt', spe_file= 'vocabs/spe_vocab/cleaned_smilesDB_1000.smi', tokenizer_type='SPE')
MacFrag_tokenizer = SMILES_SPE_Tokenizer(vocab_file='vocabs/macFrag_vocab/smilesDB_vocab.smi', tokenizer_type='MacFrag')

Bpe_tokenizer = SMILES_SPE_Tokenizer(vocab_file='vocabs/bpe_vocab/bpe_DBsmi_vocab_1000.txt', 
                                     tokenizer_path='models/tokenizers/bpe_DBsmi_1000.bin',
                                     tokenizer_type='BPE')

mofressor_tokenizer = SMILES_SPE_Tokenizer(vocab_file='./vocabs/morf_vocab/morf_tox21_vocab1000.txt',
                                           tokenizer_path='models/tokenizers/morf_tox21_1000.bin',
                                           tokenizer_type='Morfessor')

# choose tokenizer 
tokenizer = Spe_tokenizer
smi_1 = 'CS(=O)(=O)CCNCc1ccc(o1)c2ccc3c(c2)c(ncn3)Nc4ccc(c(c4)Cl)OCc5cccc(c5)F'
encoded_input = tokenizer.encode(smi_1)
print(tokenizer.tokenize(smi_1))
print(encoded_input)



['CS(=O)(=O)', 'CCN', 'Cc1ccc(', 'o1)', 'c2ccc3c(c2)', 'c(', 'ncn', '3)', 'N', 'c4cc', 'c(', 'c(', 'c4)', 'Cl)', 'OC', 'c5', 'cccc', '(', 'c5', ')', 'F']
[12, 613, 394, 264, 923, 577, 139, 549, 162, 64, 378, 139, 139, 615, 181, 165, 524, 163, 89, 524, 27, 51, 13]


In [4]:
print(mofressor_tokenizer._tokenize('CS(=O)(=O)CCNCc1ccc(o1)c2ccc3c(c2)c(ncn3)Nc4ccc(c(c4)Cl)OCc5cccc(c5)F'))
print(MacFrag_tokenizer._tokenize('CS(=O)(=O)CCNCc1ccc(o1)c2ccc3c(c2) , c(ncn3)Nc4ccc(c(c4)Cl)OCc5cccc(c5)F'))
print(Bpe_tokenizer._tokenize('CS(=O)(=O)CCNCc1ccc(o1)c2ccc3c(c2)c(ncn3)Nc4ccc(c(c4)Cl)OCc5cccc(c5)F'))
print(Spe_tokenizer._tokenize('CS(=O)(=O)CCNCc1ccc(o1)c2ccc3c(c2)c(ncn3)Nc4ccc(c(c4)Cl)OCc5cccc(c5)F'))

['C', 'S(=O)(=O)', 'C', 'C', 'N', 'C', 'c1ccc(', 'o1', ')', 'c2ccc3c(c2)', 'c(', 'ncn', '3)', 'N', 'c4cc', 'c(', 'c(', 'c4', ')', 'Cl)', 'O', 'C', 'c5c', 'ccc(', 'c5', ')', 'F']
['CS(=O)(=O)CCNCc1ccc(o1)c2ccc3c(c2) ', ' c(ncn3)Nc4ccc(c(c4)Cl)OCc5cccc(c5)F']
['CS(=O)(=O)', 'CCN', 'Cc1ccc(', 'o1)', 'c2ccc3c(c2)', 'c(', 'ncn', '3)N', 'c4ccc(', 'c(', 'c4)C', 'l)', 'OC', 'c5cccc', '(', 'c5)', 'F']
['CS(=O)(=O)', 'CCN', 'Cc1ccc(', 'o1)', 'c2ccc3c(c2)', 'c(', 'ncn', '3)', 'N', 'c4cc', 'c(', 'c(', 'c4)', 'Cl)', 'OC', 'c5', 'cccc', '(', 'c5', ')', 'F']


In [5]:
# FINE TUNING
# Modify tokenizer to add new tokens , and extend embeddings layers
# initialize the model
model_name =  "models/pre_trained_models/Best_Clintox_Spe_chemBERTa"
classification_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = Spe_tokenizer

Some weights of the model checkpoint at models/pre_trained_models/Best_Clintox_Spe_chemBERTa were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at models/pre_trained_models/Best_Clintox_Spe_chemBERTa and are newly initialized: ['classifier.out_proj.bias', 'classifier.out

In [7]:
import pandas as pd
clin_tox_zinc250 = "./datasets/cleaned_smilesDB.smi"
clin = "./datasets/pre_processed/clintox.smi"
zinc  = "./datasets/zinc_smiles.txt"
clintox_frags = './datasets/mac_fragments/clintox_fragments.smi'
# tasks, (train_df, valid_df, test_df), transformers = load_molnet_dataset("clintox", tasks_wanted=None, split='random')
# # load tox21 dataset
# tasks, (train_df, valid_df, test_df), transformers = load_molnet_dataset("tox21", tasks_wanted= ['NR-AhR'])
clintox_df = pd.read_csv('./datasets/post_processed/clintox_processed.csv')
tox21_df = pd.read_csv('./datasets/post_processed/tox21_processed.csv')
# split imbalanced dataset clintox into train, valid, test
train_df, test_df = train_test_split(clintox_df, test_size=0.2, random_state=42 , stratify=clintox_df['CT_TOX'], shuffle=True)
train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['CT_TOX'], shuffle=True)
print(train_df.shape, valid_df.shape, test_df.shape)
# Create a dictionary with the text data
tr_data_dict = {"text": train_df['smiles'].tolist() ,  #text_data
             "labels":  train_df['CT_TOX'].values}        #train_df['labels'] } #labels
val_data_dict = {"text": valid_df['smiles'].tolist() ,  #text_data
                "labels":  valid_df['CT_TOX'].values}    #train_df['labels'] } #labels

# Create a dataset using the `Dataset` class from `datasets`
tr_dataset= Dataset.from_dict(tr_data_dict)
val_dataset= Dataset.from_dict(val_data_dict) 

tr_tokenized_dataset = tr_dataset.map(lambda examples: tokenizer(examples["text"], truncation=True,padding=True, max_length=512), batched=True)
val_tokenized_dataset = val_dataset.map(lambda examples: tokenizer(examples["text"], truncation=True,padding=True, max_length=512), batched=True)
tr_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'text'])
val_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'text'])
# call add_decoded_tokens to add decoded tokens to the data
tr_tokenized_dataset = tr_tokenized_dataset.map(lambda batch: add_decoded_tokens(batch= batch, tokenizer=tokenizer))
val_tokenized_dataset = val_tokenized_dataset.map(lambda batch: add_decoded_tokens(batch= batch, tokenizer=tokenizer))
print("length of train dataset:", len(tr_tokenized_dataset))
print("length of val dataset:", len(val_tokenized_dataset))


(944, 4) (236, 4) (296, 4)


Map:   0%|          | 0/944 [00:00<?, ? examples/s]

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

Map:   0%|          | 0/944 [00:00<?, ? examples/s]

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

length of train dataset: 944
length of val dataset: 236


In [8]:
print(tr_tokenized_dataset[6])

{'text': 'CC(C)[N+](C)(CCOC(=O)C1c2ccccc2Oc3c1cccc3)C(C)C', 'labels': tensor(0), 'input_ids': tensor([ 12, 236,  94, 138, 161, 401, 149, 201,  30, 145, 153, 162, 581,  13,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'decoded_tokens': '[CLS] CC(C) [N+] (C) (CC OC(=O) C1 c2ccccc2 O c3 c1cccc 3) C(C)C [SEP] [PAD] [PAD] [PAD] 

In [13]:
# Train the model

data_collator= DataCollatorWithPadding(tokenizer=tokenizer, padding ="longest", max_length=128)
train_loader = DataLoader(tr_tokenized_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_tokenized_dataset, batch_size=16, shuffle=True)
# load metric for binary classification for imbalanced dataset
roc_auc_score = evaluate.load("roc_auc")
# accuracy = load_metric("accuracy")
model_name = "clintox_spe_classifier_chemBERTa"
training_args = TrainingArguments(
    output_dir="./models/classifiers/"+model_name,
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    save_steps=10000,
    save_total_limit=1,
    logging_steps=5,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    seed=42,
    load_best_model_at_end=True,
    # metric_for_best_model=roc_auc_score.name,
    )

# Calculate class weights
print("labels counts " , np.unique(clintox_df["CT_TOX"], return_counts=True))
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(clintox_df["CT_TOX"], return_counts=True)[0], y=clintox_df["CT_TOX"])
class_weights = torch.tensor(class_weights, dtype=torch.float)
print("class weights ", class_weights)

# Define the Trainer
trainer = ToxicTrainer(
    model=classification_model,
    args=training_args,
    train_dataset=tr_tokenized_dataset,
    eval_dataset=val_tokenized_dataset,
    data_collator=data_collator,
    compute_metrics= lambda predictions: compute_metrics(predictions, class_weights),
    weights=class_weights,
)

labels counts  (array([0, 1]), array([1364,  112]))
class weights  tensor([0.5411, 6.5893])


In [14]:
# apply torch.optim.AdamW instead
optimizer = torch.optim.AdamW(classification_model.parameters(), lr=1e-4)
trainer.optimizer = optimizer
# Train the model
trainer.train()
# save model
trainer.save_model("./models/classifiers/best_"+model_name)

  0%|          | 0/15 [00:00<?, ?it/s]

{'loss': 0.6896, 'learning_rate': 6.666666666666667e-05, 'epoch': 0.33}
{'loss': 0.6568, 'learning_rate': 3.3333333333333335e-05, 'epoch': 0.67}
{'loss': 0.6337, 'learning_rate': 0.0, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.592046856880188, 'eval_accuracy': 0.8569678159192854, 'eval_precision': 0.8754064674119615, 'eval_recall': 0.8333333333333334, 'eval_f1': 0.8568942986915573, 'eval_balanced_accuracy_score': 0.857033639143731, 'eval_AUC': 0.8570336391437311, 'eval_runtime': 20.6781, 'eval_samples_per_second': 11.413, 'eval_steps_per_second': 0.193, 'epoch': 1.0}
{'train_runtime': 364.524, 'train_samples_per_second': 2.59, 'train_steps_per_second': 0.041, 'train_loss': 0.6600124994913737, 'epoch': 1.0}


In [22]:
trainer.evaluate()

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.592046856880188,
 'eval_accuracy': 0.8569678159192854,
 'eval_precision': 0.8754064674119615,
 'eval_recall': 0.8333333333333334,
 'eval_f1': 0.8568942986915573,
 'eval_balanced_accuracy_score': 0.857033639143731,
 'eval_AUC': 0.8570336391437311,
 'eval_runtime': 21.1793,
 'eval_samples_per_second': 11.143,
 'eval_steps_per_second': 0.189,
 'epoch': 1.0}